In [47]:
import pandas as pd
from tqdm import tqdm
import json
import os

In [2]:
os.chdir('/Users/KarolinaNaranjo/Library/Mobile Documents/com~apple~CloudDocs/DS/cc/annotated_data')

### Combine Dataframes

In [3]:
# get an unified dataframe with all documents
df_test1_docs = pd.read_csv('documents_df_2023-12-03.csv')
df_test2_docs = pd.read_csv('documents_df_2023-12-06.csv')

df_test1_format = pd.read_csv('paragraphs_df_2023-12-03.csv')
df_test2_format = pd.read_csv('paragraphs_df_2023-12-06.csv')

df_test1_semantic = pd.read_csv('styles_df_2023-12-03.csv')
df_test2_semantic = pd.read_csv('styles_df_2023-12-06.csv')

In [21]:
def combine_dataframes(docs1_path, docs2_path, format1_path, format2_path, semantic1_path, semantic2_path):

    # get an unified dataframe with all documents
    df_test1_docs = pd.read_csv(docs1_path)
    df_test2_docs = pd.read_csv(docs2_path)

    df_test1_format = pd.read_csv(format1_path)
    df_test2_format = pd.read_csv(format2_path)

    df_test1_semantic = pd.read_csv(semantic1_path)
    df_test2_semantic = pd.read_csv(semantic2_path)

    # get length of each one of the first dataframes (the ones to add something to)
    len_test1_docs = len(df_test1_docs)
    len_test1_format = len(df_test1_format)
    len_test1_semantic = len(df_test1_semantic)

    # offset df_test2_docs id by len_test1
    df_test2_docs['id'] = df_test2_docs['id'] + len_test1_docs


    # offset df_test2_format "document" by len_test1
    df_test2_format['document'] = df_test2_format['document'] + len_test1_docs
    #offset df_test2_format "id" by len_test1_format
    df_test2_format['id'] = df_test2_format['id'] + len_test1_format


    # offset df_test2_semantic "document" by len_test1
    df_test2_semantic['document'] = df_test2_semantic['document'] + len_test1_docs    
    #offset df_test2_semantic "id" by len_test1_semantic
    df_test2_semantic['id'] = df_test2_semantic['id'] + len_test1_semantic


    # concatenate df_test1_docs and df_test2_docs, df_test1_format and df_test2_format, df_test1_semantic and df_test2_semantic
    df_docs = pd.concat([df_test1_docs, df_test2_docs], ignore_index=True)
    df_format = pd.concat([df_test1_format, df_test2_format], ignore_index=True)
    df_semantic = pd.concat([df_test1_semantic, df_test2_semantic], ignore_index=True)

    df_docs = df_docs[['id', 'name', 'type', 'year', 'num_sections', 'num_paragraphs']]
    df_format=df_format[['id', 'document', 'paragraph_id', 'previous', 'next','length', 'document_pos', 'section', 'function', 'text']]
    df_semantic=df_semantic[['id', 'document', 'paragraph_id', 'text_type', 'length', 'document_pos', 'paragraph_pos', 'function', 'text', 'section']]

    return df_docs, df_format, df_semantic


In [22]:
df_docs, df_format, df_semantic = combine_dataframes('documents_df_2023-12-03.csv', 'documents_df_2023-12-06.csv', 
                                                     'paragraphs_df_2023-12-03.csv', 'paragraphs_df_2023-12-06.csv', 
                                                     'styles_df_2023-12-03.csv', 'styles_df_2023-12-06.csv')

In [23]:
# df_docs.to_csv('df_docs.csv', index=False)
# df_format.to_csv('df_format.csv', index=False)
# df_semantic.to_csv('df_semantic.csv', index=False)

In [40]:
format_test1 =df_format.sample(frac=1)

In [42]:
# split into train, val, test
train= format_test1[:int(len(format_test1)*0.7)]
val=df_format[int(len(format_test1)*0.7):int(len(format_test1)*0.9)]
test=df_format[int(len(format_test1)*0.9):]

### Fine-tuning data files

In [24]:
def df_to_json(df_format,target='section',prompt="prediga a que sección pertenece el siguiente texto de la sentencia de la corte constitucional"):
    init_prompt=prompt
    output=[]
    for i in tqdm(df_format.index):
        output.append({'instruction':init_prompt,
                       "input":df_format.loc[i,'text'],
                       "output":df_format.loc[i,target]}
                       )
    return output

In [25]:
%%time
result=df_to_json(df_format,target='section',prompt="prediga a que sección pertenece el siguiente texto de la sentencia de la corte constitucional")

100%|██████████| 388439/388439 [00:04<00:00, 88293.30it/s]


CPU times: user 5.12 s, sys: 136 ms, total: 5.26 s
Wall time: 5.26 s


In [43]:
train1_json=df_to_json(train)
val1_json=df_to_json(val)
test1_json=df_to_json(test)

100%|██████████| 38844/38844 [00:00<00:00, 91716.86it/s]


In [44]:
# save json data
with open('train1.json','w') as jsonfile:
    json.dump(train1_json,jsonfile)

In [45]:
with open('val1.json','w') as jsonfile:
    json.dump(val1_json,jsonfile)

In [46]:
with open('test1.json','w') as jsonfile:
    json.dump(test1_json,jsonfile)

In [13]:
#get text where index is 0
df_format.loc[3,'text']

3     Demanda de inconstitucionalidad de los artícu...
3     Se reiterará la jurisprudencia de esta Corpor...
Name: text, dtype: object

In [ ]:
# Previous code to get the dataframes

# data1=pd.read_csv('paragraphs_df_2023-12-03.csv')
# data2=pd.read_csv('styles_df_2023-12-03.csv')
# big_dataset=pd.concat([data1,data2],ignore_index=True)
# # sample dataset
# big_dataset=big_dataset.sample(frac=1)
# big_dataset=pd.concat([data1,data2],ignore_index=True)
# del big_dataset['Unnamed: 0']

In [ ]:
data1['section'].value_counts()

In [ ]:
# def df_to_json(data1,target='section',prompt="predice la sección de la decisión judicial de la corte constitucional"):
#     init_prompt=prompt
#     output=[]
#     for i in tqdm(data1.index):
#         output.append({'instruction':init_prompt,"input":data1.loc[i,'text'],"output":data1.loc[i,target]})
#     return output
        
    

In [ ]:
# result=df_to_json(train)

In [ ]:
# # save json data
# with open('train.json','w') as jsonfile:
#     json.dump(result,jsonfile)

In [ ]:
# results2=df_to_json(data2,target='text_type',prompt="detecte la anotación semántica del siguiente texto")

In [ ]:
# with open('semantic_annotation.json','w') as jsonfile:
#     json.dump(results2,jsonfile)

In [ ]:
# with open('format_annotation.json','r') as gfile:
#     content=json.load(gfile)

In [ ]:
# content